In [1]:
import nltk
#from nltk.lemmatize.lancaster import Lancasterlemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle
import speech_recognition as sr

r = sr.Recognizer()

from gtts import gTTS
import os


C:\Anaconda\envs\chat\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\envs\chat\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\envs\chat\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda\envs\chat\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [4]:
with open("intents.json") as file:
    data = json.load(file)
ch=1


In [5]:
if ch!=1:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
else:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [lemmatizer.lemmatize(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [lemmatizer.lemmatize(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.reset_default_graph()

In [6]:
net = tflearn.input_data(shape=[None, len(training[0])],name='chat_in')
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]),activation="softmax",name='chat_out')
net = tflearn.regression(net)

model = tflearn.DNN(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [7]:
try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Raghav N G\Downloads\chatbot\model.tflearn


In [8]:
with net.graph.as_default():
    del tensorflow.get_collection_ref(tensorflow.GraphKeys.TRAIN_OPS)[:]
 
    
model.save("model.tflearn")


INFO:tensorflow:C:\Users\Raghav N G\Downloads\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
with tensorflow.Session() as session:
    my_saver = tensorflow.train.import_meta_graph('model.tflearn.meta')
    my_saver.restore(session, tensorflow.train.latest_checkpoint('.'))
    
    # Rest of the code goes here
    output_node_names = [n.name for n in tensorflow.get_default_graph().as_graph_def().node]
    #print(output_node_names)
    frozen_graph = tensorflow.graph_util.convert_variables_to_constants(
    session,
    session.graph_def,
    ['chat_out/Softmax']
    )
    
    with open('frozen_model.pb', 'wb') as f:
        f.write(frozen_graph.SerializeToString())



INFO:tensorflow:Restoring parameters from C:\Users\Raghav N G\Downloads\chatbot\model.tflearn
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


In [11]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        with sr.Microphone() as source:
            audio = r.listen(source,phrase_time_limit=5)

    
        try:
            speech_to_text=r.recognize_google(audio,language='en-US')
            print("YOU:"+speech_to_text)
        
            #print("You:")
            inp = speech_to_text
            if inp.lower() == "quit":
                break
    
            results = model.predict([bag_of_words(inp, words)])
            results_index = numpy.argmax(results)
            tag = labels[results_index]
    
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
    
            #print(random.choice(responses))
            
            text_to_speech=random.choice(responses)
            print(text_to_speech)
            tts = gTTS(text=text_to_speech, lang='en')
            tts.save("C:/Users/Raghav N G/Downloads/chatbot/intro.mp3")

            os.system("start intro.mp3")
        except:
            pass

In [12]:
chat()

##########

#The results seen under YOU: are actually the speech given to the model, which prints the received text and passes the text to the model for further processing.

##########

Start talking with the bot (type quit to stop)!
YOU:find
any other remedies taken
YOU:no
Dont worry bhaiya!! If medication taken: make sure you have had enough sleep or eat properly.     i'll suggest some if you have not taken: take paracetamol or crocin if you feel feverish. If you feel dizzy or tired take enough water and some sleep.
YOU:how old are you
18 years young!
YOU:quit
